In [1]:
import parsl
import os
import logging
from tqdm import tqdm
import json
import time

import numpy as np

from parsl.app.app import python_app

from coffea.util import save, load
from coffea import hist
from coffea.processor import run_parsl_job
from coffea.processor.parsl.parsl_executor import parsl_executor

chunksize=500000

In [2]:
# from configs.unl_condor import config
# from configs.unl_slurm import config
from configs.notre_dame import config

parsl.load(config)

In [3]:
with open('metadata/samplefiles.json') as f:
    datasets = json.load(f)['Hbb_2017']

In [4]:
start = time.time()
treenames = ['otree', 'Events']  # deal with mixed skims and full derived trees
final_accumulator = run_parsl_job(
    datasets,
    treenames,
    load('boostedHbbProcessor.coffea'), # the hbb analysis worker
    parsl_executor, 
    executor_args={'config': None},
    chunksize=chunksize
)
dt = time.time() - start

In [5]:
num_events = sum(parsl_executor.counts.values())
nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
nfilled = sum(sum(np.sum(arr>0) for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))

print('processed: {:,d} events'.format(num_events))
print('total time: {:.2f} min'.format(dt / 60))
print('{:.2f} μs/event'.format(dt / num_events * 1e6))
print('{:.2f} Mevent/s'.format(num_events / dt / 1e6))

print("filled {:.1f} bins".format(nbins/ 1e6))
print("nonzero bins: {:.1f}%".format(100 * nfilled / nbins))


processed: 290,837,857 events
total time: 7.65 min
1.58 μs/event
0.63 Mevent/s
filled 99.2 bins
nonzero bins: 1.9%


In [ ]:
save(final_accumulator, 'hists.coffea')

In [ ]:
parsl.dfk().cleanup()
parsl.clear()

In [6]:
import subprocess